# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Big Data Real-Time Analytics com Python e Spark</font>

# <font color='blue'>Capítulo 6 - </font>

# <font color='blue'>Machine learning em python</font>

In [ ]:
import sklearn as sl
import warnings
warnings.filterwarnings('ignore')

## Definição do problema de negocio

Criar um modelo preditivo que possa prever se uma pessoa pode ou não desenvolver diabetes

Dataset: Pima Indians Diabetes dataset

http://archives/ics.uci.com/ml/datasets/diabetes

Registros medicos de pacientes do Pima Indians e cada registro está marcado se paciente teve ou nao diabetes

Variáveis:
- numero de vezes gravida
- concentração de glicose em duas horas em um teste oral de tolerancia à glicose
- pressão sanguinea (mm Hg)
- espessura da dobra cutânea do triceps (mm)
- 2 horas de insulina 
- indice de massa corporal
- linhagem de diabetes
- idade (anos)
- label (variavel target) 0/1

# Extraindo e carregando dados

Existem diversar considerações ao se carregar dados para processo de machine learning. ex, seus dados possui um header (cabeçalho)? se nao possuir, voce precisa definir o titulo de cada coluna, seus arquivos possuem comentarios?, qual o delimitador de cada linha. alguns dados estão entre aspas, simples ou duplas?

In [ ]:
# carregando arquivo csv usando Numpy
import numpy as np
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
prima_data = open(arquivo, 'rb')
prima_dados = np.loadtxt(prima_data, delimiter=',')
print(prima_dados.shape)

In [ ]:
# carregando arquivo csv com Pandas
import pandas as pd
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = pd.read_csv(arquivo, names=colunas)
print(dados.shape)

## Análise exploratória dos dados

estatistica descritiva

In [ ]:
# verificando as primeiras 20 linhas
dados.head(20)

Se o numero de linhas for muito grande, o algoritmo pode levar muito tempo para ser treinado. se o numero de registros for muito pequeno, voce pode não ter muitos registros para o treino do seu modelo

Se você tiver muitas colunas em seu arquivo, o algoritmo pode apresentar problema de performance devido a alta dimensionalidade.

A melhor solução vai depender de cada caso, mas lembrese, treine seu modelo em um subset do seu conjunto de dados maior e depois aplique o modelo a novos dados.

In [ ]:
# visualisando as dimenões
dados.shape

O tipo de dados é muito importante. Pode ser necessário converter strings, ou colunas com numeros inteiros podem representar variáveis categóricas ou valores ordinários

In [ ]:
# tipo de dados de cada atributo
dados.dtypes

In [ ]:
# sumário estatistico
dados.describe()

Em problemas de classificação pode ser necessário balancear as classes. Classes desbalanceadas (volume maior de um dos tipos da classe) são comuns e precisam ser tratados durante a fase de pre-processamento. Podemos ver a baixo que existe uma clara desproporção entre as classes 0 (não ocorrencia de diabetes) e 1 (ocorrencia de diabetes)

In [ ]:
dados.groupby('class').size()

A correlação é o relacionamento entre duas variáveis. O método mais comum para calcular a correlação é o método de "pearson" que assume uma distribuição normal dos dados.Correlação de -1 mostra uma correlação negativa, enquanto uma correlação de +1 mostra uma correlação positiva. uma correlação igual a 0 mostra que não á uma correlação entre as variáveis

Alguns algoritmos como regreção linear e regreção logistica podem apresentar problemas deperformance se ouver atributos altamente correlacionados (colineares)

In [ ]:
dados.corr('pearson')

Skew (ou simetria) se refere a distribuição de dados que é assumida ser normal ou gaussiana (bell curve). Muitos algoritmos de machine learning consideram que os dados possuem uma distribuição normal. Conhecendo a assimetria dos dados, permite que Você faça uma preparação e entregue oque o algoritmo espera receber. aumentando dessa forma a acuracia do modelo

In [ ]:
dados.skew()

## Visualização com matplotlib

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Com o histograma podemos rapidamente avaliar a distribuição dos dados para cada atributo. OS histrogramas agrupam os dados em bins e fornecem uma contagem do numero de observações em cada bin. COm o histograma, você pode rapidamente verificar a assimetria  dos dados e se eles estão em uma distribuição normal ou não. isso também vai ajudar na identificação de outliers.

Podemos ver que os atributos age, ped e test possuem uma distribuição exponencial. Podemos ver que as colunas mass e press possuem uma distribuição normal 

In [ ]:
dados.hist()
plt.show()

Os density plots são uma outra forma de visualizar a distribuição dos dados para cada atributo. O plot é uma especia de histograma abstrato com ua curva suave atravez do topo dos bins. de um histograma. Pode ser mais facil identificar a distribuição dos dados usando density plots

In [ ]:
# density plots univariado
dados.plot(kind='density', subplots=True, layout=(3,3), sharex = False)
plt.show()

Com os boxplots também podemos revisar a distribuição dos dados para cada atributo. A linha no centro (vermelho) é o valor da mediana (quantil 50%), a linha a baixo é o quantil 25% e a linha a cima é o quantil 75%. O boxplot ajuda a ter uma idéia da dispersão dos dados e os possiveis outliers

Podemos ver a dispersão dos dados é bem diferente entre os atributos. As colunas Age, skin e test possuem uma simetria muito proxima a valores de dados menores

In [ ]:
dados.plot(kind='box', subplots=True, layout=(3,3), sharex = False)
plt.show()

In [ ]:
# matriz de correlação com nome das variaveis

correlation = dados.corr()
import numpy as np
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlation, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,9,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(colunas)
ax.set_yticklabels(colunas)
plt.show()

Um scatterplot mostra o relacionamento entre duas variaveis como pontos em duas dimensões, sendo um eixo para cada variavel. Podemos criar um scatterplot para cada par de variaveis do dataset. a exemplos da matriz de correlação, o scatterplot matrix é simetrico

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(dados)
plt.show()

## Visualização com seaborn

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(dados)

In [ ]:
sns.boxplot(data=dados, orient='v')

In [ ]:
sns.clustermap(dados)

In [ ]:
dados.describe()

In [ ]:
from scipy import stats
sns.distplot(dados.pedi, fit = stats.norm)

## Preparando os dados para machine learning

Muitos algoritmos esperam receber os dados em um formato especifico. É seu trabalho preparar os dados em uma estrutura que seja adequada ao algoritmo que você esteja utilizando

É muito provavel que você tenha que realizar tarefas de pré-processamento nos dados. Esse é um passo necessário no processamento. O desafio é fato que cada algotirmo tem uma estrutura diferente, ou pode requerer diferentes transformações nos dados. Más é possivel, em alguns casos, obter bons resultados sem um trabalho de pre-processamento. Más é uma boa pratica criar diferentes visoes e transformações nos dados, de modo a poder testar diferentes algoritmos de machine learning. 

## Normalização - Método 1

É uma das primeiras tarefas de pré-processamento, é colocar os dados na mesma escala. Muitos algoritmos de machine learning vão se beneficiar disso e produzir resultados melhores. Significa colocar os dados em uma escala entre 0 e 1. Isso é util para otimização, sendo usado no core de MachineLearning, como gradient descent. Isso é util tambem para algoritmos como regreção, redes neurais e algoritmos que usam medidas de distancia, como o KNN. O scikit-learn possui uma função para essa etapa chamado MinMaxScaler()

In [ ]:
# Transformando os dados em uma mesma escala, entre 0 e 1

# Import dos modulos
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Carregando os dados

arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

In [ ]:
# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

In [ ]:
# gerando nova escala (Normalizando os dados)
scaler = MinMaxScaler(feature_range=(0,1))
rescaledX = scaler.fit_transform(x)

In [ ]:
# sumarizando os dados transformados
print("Dados originais \n\n", dados.values)
print("Dados normalizados \n\n", rescaledX[0:5,:])

## Normalização método 2

Normalização se refere a ajustar a escala de cada observação (linha) de modo que ela tenha comprimento igual a 1 (chamado de vetor de comprimento 1 em algebra linear). Esse método de pré processamento é util quando temos datasets esparsos (com muitos zeros), e atributos com escalas muito variada. Util quando utilizamos algoritmos de redes neurais ou que usam medidas de distancia. como o KNN. O scikt-leanr possui uma função chamada Normalizer()

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
scaler = Normalizer().fit(x)
normalizedX = scaler.transform(x)

In [ ]:
# sumarizando os dados transformados
print("Dados originais \n\n", dados.values)
print("Dados normalizados \n\n", normalizedX[0:5,:])

## Padronização

Padronização é a técnica de transformar os dados com distribuição Gaussiana (normal) e diferentes médias e desvios padrão em uma distribuição Gaussiana com média 0 e desvio padrão 1. Isso é util para algoritmos que esperam que os dados estejam com uma distribuição Gaussiana, como regreção linear, regreção logistica e linear discriminant analisys. funciona bem quando os dados já estão na mesma escala. O scikt-leanr possui uma função chamada StandardScaler()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler().fit(x)
standardX = scaler.transform(x)

In [ ]:
# sumarizando os dados transformados
print("Dados originais \n\n", dados.values)
print("Dados padronizados \n\n", standardX[0:5,:])

## Binarização (Transformação dos dados e mvalores binarios)

Nós podemos definir um valor em nossos dados, ao qual chamamos de threshold e então definimos que todos os valores a cima do threshold serão marcados como sendo 1 e todos os valores iguais ou menores serão marcados como 0. Isso é oque chamamos de binarização. Isso é util quando temos probabilidades e queremos transformar os dados em algo com mais significado. O scikt-learn possui a função Binarizer()

In [ ]:
from sklearn.preprocessing import Binarizer

In [ ]:
binarizer = Binarizer(threshold=0.2).fit(x)
binarizedX = binarizer.transform(x)

In [ ]:
# sumarizando os dados transformados
print("Dados originais \n\n", dados.values)
print("Dados binarizados \n\n", binarizedX[0:5,:])

## Feature Selection

Os atributos presentes no seu dataset e que você utiliza nos dados de treino, terão grande influencia na precisão e resultado do seu modelo preditivo. Atributos irrelevantes terão impacto negativo na performance. enquanto atributos colineares podem afetar o grau de acuracia do modelo. O scikt-learn possui funções que automatizam o trabalho de extração e seleção de variaveis

A etapa de feature selection é onde selecionamos os atributos (variáveis) que serão melhores candidatas a variaveis preditoras. O feature selection nos ajuda a reduzir o overfitting (quando o algoritmo aprende demais), aumenta a acurácia do modelo e reduz o tempo de treinamento

## Seleção univariada

Testes estatisticos podem ser usados para selecionar os atributos que possuem forte relacionamento com a variável que estamos tentando prever. O scikt-learn fornece a função SelectKBest() que pode ser usada com diversos testes estatisticos, para selecionar os atributos. vamos usar o teste qui-quadrado e selecionar as 4 melhores variaveis que possam ser usadas como variaveis preditoras.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
#função para seleção de variaveis
best_var = SelectKBest(score_func=chi2, k=4)

# executa a função de pontuação (em X e Y) e obtem os recursos selecionados
fit = best_var.fit(x,y)

# Reduz x para os recursos selecionados
features = fit.transform(x)

In [ ]:
# Resultado
print("\nNumero original de features: ", x.shape[1])
print("\nNumero reduzido de features: ", features.shape[1])
print("\nVariáveis selecionadas", features)

## Eliminação recursiva de atributos

Essa é outra tecnica para seleção de atributos, que recursivamente remove os atributos e constrói o modelo com os atributos remanecentes. Esta técinca utiliza acurácia do modelo para identificar os atributos que mais contribuem para prever a variável alvo. Em ingles essa técnica significa Recursive Feature Elimination FRE

O exemplo a baixo utiliza a técnica de eliminação recursiva de atributos com um algoritmo de regreção logistica para selecionar as 3 melhores variáveis preditoras. O FRE selecionou as variaveis preg, mass e pedi que estão marcadas como true em "Atributos selecionados" e com valor 1 em "ranking dos atributos"

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
modelo = LogisticRegression()

rfe = RFE(modelo, 3)
fit = rfe.fit(x, y)

In [ ]:
# print dos resultados
print("\nNumero de atributos: ", fit.n_features_)
print("\nVariáveis preditoras: ", dados.columns[0:8])
print("\nVariaveis selecionadas: ", fit.support_)
print("\nRanking dos atributos: ", fit.ranking_)

## Método Ensamble para seleção de variávis

Bagged Decision Trees, como o algoritomo Random Forest (esses são chamados de métodos Emsamble) podem ser usados para estimar a importãncia de cada atributo. Esse método retorna um score para cada atributo.

Quanto maior o score maior a importancia do atributo

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
modelo = ExtraTreesClassifier()
modelo.fit(x,y)

In [ ]:
print("\nVariáveis preditoras: ", dados.columns[0:8])
print("\nScore de variaveis:",modelo.feature_importances_)

## Redução de dimencionalidade (Feature Extraction)

# Principal Component Analisys (PCA)

o PCA foi inventado em 1901 por Karl Pearson e utiliza agebra linear para transformar datasets em uma forma comprimida, o que é conhecido como redução de dimencionalidade. Com PCA você pode escolher o numero de dimensões (na forma de componente) a partir de um grande conjunto de variáveis disponíveis em um conjunto de dados. Esta técnica permite extrair um numero pequeno de conjuntos dimensionais a partir de um dataset altamente dimensional. Com menos variaveis a visualização também se torna muito mais significativa. PCA é mais util quando se lida com 3 ou mais dimensões

Cada componente resultante é uma combinação linear de n atributos. Ou seja, cada componente principal é a combinação de componentes dentro do dataset. O primeiro componente principal é a combinação linear dos atributos com máxima variãncia e determina a direção a direção em que hà mais alta variabilidade nos dados. Quanto maior a variabilidade capturada no primeiro componente principal, mais informação será capturada pelo componente, o segundo componente principal captura a variabilidade remanecente. Todos os componentes remanescentes seguem o mesmo conceito.

O PCA precisa ser alimentado com dados normalizados. Utilizar o PCA em dados não normalizados pode gerar resultados inesperados.

A Análise de componentes principais é uma tecnica da estatistica multivariada que consiste em transformar um conjunto de variáveis originais em outro conjunto de variáveis denominadas de componentes principais. Os componentes principais apresentam propriedades importantes. cada componente principal é uma combinação linear de todas as variáveis originais. São independentes entre si e estimados com o proposito de reter, em ordem de estimação, o máximo de informação, e mtermos da variação contida dos dados. Os componentes principais são garantidamente independentes apenas se os dados forem normalmente distribuidos

Procura-se redistribuir a variaão observada nos eixos originais, de forma a se obter um conjunto de eixos ortogonais não correlacionados. Esta técinca pode ser utilizada par geração de indices e agrupamento de individuos. A análise agrupa os individuos de acordo com sua variação, isto é os individuos são agrupados segundo sua variãncia, ou seja, segundo seu comportamento dentro da população, representado pelo conjunto de caracteristicas que define o individuo, ou seja a atécnica agrupa os individuos, segundo a variação de suas caracteristicas.

A análise de componentes principais é associada a idéia de redução de massa de dados, com menor perda possivel de informação

Objetivo é sumarizar os dados que contem muitas variáveis (p) por um conjunto menor de variáveis (k). compostas derivadas a partir do conjunto original. PCA usa um conjunto de dados representados por uma matriz de n registros por p atributos que podem estar correlacionados e sumariza esse conjunto por eixos não correlacionados. que são uma combinação linear das p variáveis originais. as primeiras k variáveis contém a maior quantidade de variação dos dados.

Em termos gerais o PCA busca reduzir o numero de dimensões de um dataset, projetando os dados em um novo plano. Usando essa nova projeção os dados originais, que podem envolver diversas variáveis, podem ser interpretados usando menos dimenões

No dataset reduzido podemos observar com mais clareza tendencias, padrões e/ou outliers. Mas vale lembrar que a regra "se não está nos dados brutos, não existe" é sempre valida. A PCA fornece apenas mais claresa as infirmações que já estão lá


In [ ]:
from pandas import read_csv
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

In [ ]:
scaler  = MinMaxScaler(feature_range=(0,1))
rescaledx = scaler.fit_transform(x)

In [ ]:
pca = PCA(n_components=4)
fit = pca.fit(rescaledx)

In [ ]:
# sumarixando os componentes
print("Variância %s" % fit.explained_variance_)
print("\nComponents",fit.components_)

## Amostragem - Resampling

Você precisa saber se seu modelo preditivo vai funcionar bem quando receber novos dados. A melhor mandeira de validar a performance do modelo é fazer previsões em dados que você já conhece o resultado. Outra maneira de testar a performance do seu modelo é utilizar técnicas de estatistica como método de amostragem que permite você estimar quão bem seu modelo irá fazer previsões em novos dados.

A avaliação do modelo é uma estatistica de quão bem o algoritmo será capaz de prever em novos dados. Isso não garante performance. Após avaliar o modelo, nós podemos treinar o modelo novamente com os dados de treino e então preparalo para o uso operacional em produção. Existem diversas técnicas para isso e estudaremos duas, conjunto de dados de treino e teste e, cross-validation

# Dados de treino e de teste

Este é o método mais utilizado para avaliar a performance de um algoritmo de machine learning. dividimos os dados originais em dados de treino e de teste. treinamos o algoritmo nos dados de treino e, fazemos as previsões nos dados de teeste e avaliamos o resultado. A divisão dos dados vai depender do seu dataset, más utiliza se com frequencia tamanhos entre 70/30 e 65/35 (treino e teste)

Este método é bem veloz e ideal para conjunto de dados muito grandes. O ponto negativo é a alta variancia

In [ ]:
# avaliação usando dados de treino e teste

from pandas import read_csv
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

# definindo o tamanho das amostras 
teste_size = 0.33

# garante que os resultados podem ser reproduzidos
# isso é importante para comparar a acuracia com outros algoritmos de machine learning
seed = 7

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
# criando conjunto de dados de treino e teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(x,y, test_size = teste_size, random_state = seed)

# criação do modelo
modelo = LogisticRegression()

# treinamento do modelo
modelo.fit(X_treino, Y_treino)

In [ ]:
result = modelo.score(X_teste, Y_teste)
print("Acurácia nos dados de teste: %.3f%%" % (result * 100.0))

## Cross Validation

Cross validation é uma tecnica que pode ser utilizada para avaliar a performance de um modelo com menos variancia do que a tecnica de dividir os dados em treino/teste. com essa tecnica dividims em partes normalmente chamadas de k-folds. Cada parte é chamada fold. Podemos sumarizar a performance em cada fold usando a média e o desvio padrão. O resultado normalmente é mais confiavel, e oferece maior acuracia ao modelo. A chave desse processo está em definir o correto valor de k, de modo que o numero de folds represente adequadamente o numero de repetições necessárias.

In [ ]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:
# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

# definindo o tamanho das amostras 
seed = 7
num_folds = 5

In [ ]:
kfold = KFold(num_folds, True, random_state=seed)
model = LogisticRegression()
resultado = cross_val_score(model, x, y, cv=num_folds)

print("Acurácia Final %.3f%%" % (resultado.mean() * 100.0))

## Avaliando performance do modelo

As métricas que você escolhe para avaliar a performance do seu modelo vão influenciar a forma como a performance é medida e comparada com modelos criados com outros algoritmos

# Métricas para algoritmos de classificação



In [ ]:
# Acurácia

# numero de previsões corretas. É util apenas quando existe o mesmo numero de observações em cada classe.
resultado = cross_val_score(model, x, y, cv=num_folds, scoring='accuracy')

print("Acurácia Final %.3f%%" % (resultado.mean() * 100.0))


# Curva ROC

A curva ROCK permite analisar as metricas AUC (UNDER THE CURVE)
Essa é uma métrica de performance para classificação binária, em que podemos definir as classes em positivas e negativas
Problemas de classificação binária são um trade-off entre sensitivity e Specifity
Sensitivity é a taxa de verdadeiro positivo TP. esse é o numero de instancias positivas da primeira classe que foram previstas
Specifity é a taxa de verdadeiro negativo TN. esse é o numero de instancias negativas da segunda classe que foram previstas
Valores a cima de 0.5 indicam uma boa taxa de previsão

In [ ]:
resultado = cross_val_score(model, x, y, cv=num_folds, scoring='roc_auc')

print("Acurácia Final %.3f%%" % (resultado.mean() * 100.0))

In [ ]:
# Confusion matrix
# Permite verificar a acurácia atravez de uma matrix

# criando conjunto de dados de treino e teste
from sklearn.model_selection import train_test_split
X_treino, X_teste, Y_treino, Y_teste = train_test_split(x,y, test_size = teste_size, random_state = seed)

# criação do modelo
modelo = LogisticRegression()

# treinamento do modelo
modelo.fit(X_treino, Y_treino)

previsoes = modelo.predict(X_teste)

from sklearn.metrics import confusion_matrix
confusion_matrix(previsoes, Y_teste)

In [ ]:
# relatório de clasificação
from sklearn.metrics import classification_report

print(classification_report(Y_teste, previsoes))

## Algoritmos de classificação

Não temos como saber qual algoritmo vai funcionar melhor na constução do modelo, antes de testarmos o algoritmo no dataset. O ideal é testar alguns algoritmos e então escolher o que oferece melhor nivel de precisão. Vamos testar um conjunto de algoritmos de classificação. nas mesmas condições

# Regreção logistica

Algoritmo Linear. O Algoritmo de regreção logistica assume que seus dados estão normalizados (Em uma distribuição normal) para valores numéricos que podem ser modelados com classificação binária

In [ ]:
from pandas import read_csv

# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

num_folds = 10
seed = 7

from sklearn.model_selection import KFold
kfold = KFold(num_folds, True, random_state=seed)

from sklearn.linear_model import LinearRegression
model = LinearRegression()

from sklearn.model_selection import cross_val_score
resultado = cross_val_score(model, x, y, cv=kfold)

print("Acurácia Final %.3f%%" % (resultado.mean() * 100.0))

## Linear discriminant Analisys

Algoritmo linear. Técnica estátistica par aclassificação binária. Também assume que os dados estão em uma distribuição normal

In [ ]:
from pandas import read_csv

# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

num_folds = 5
seed = 7

from sklearn.model_selection import KFold
kfold = KFold(num_folds, True, random_state=seed)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis()

from sklearn.model_selection import cross_val_score
resultado = cross_val_score(model, x, y, cv=kfold)

print("Acurácia Final %.3f%%" % (resultado.mean() * 100.0))

## KNN K-nearest Neighbors

Algoritmo não linear que utiliza uma métrica de distancia para encontrar o valor de k mais adequado as distãncias do dataset de treino

In [ ]:
from pandas import read_csv

# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

num_folds = 5
seed = 7

from sklearn.model_selection import KFold
kfold = KFold(num_folds, True, random_state=seed)

from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()

from sklearn.model_selection import cross_val_score
resultado = cross_val_score(model, x, y, cv=kfold)

print("Acurácia Final %.3f%%" % (resultado.mean() * 100.0))

## Naive Bayes

Algoritmo não linear. Calcula a probabilidade de cada classe e a probabilidade condicional de cada classe dado uma variavel de entrada. As probabilidades são então estimadas para o novo dado e multiplicadas. assumindo que são independentes (suposição simples ou naive). Assume dados em distribuição Gaussiana (normal)

In [ ]:
from pandas import read_csv

# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

num_folds = 5
seed = 7

from sklearn.model_selection import KFold
kfold = KFold(num_folds, True, random_state=seed)

from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

from sklearn.model_selection import cross_val_score
resultado = cross_val_score(model, x, y, cv=kfold)

print("Acurácia Final %.3f%%" % (resultado.mean() * 100.0))

## CART (Classification and Regretion tree)

Algoritmo não linear. O algoritmo CART constrói uma arvore binária do dataset de treino. Cada atributo e cada valor de cada atributo são avaliados com o objetivo de reduzir a função de custo (Cust function)

In [ ]:
from pandas import read_csv

# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

num_folds = 5
seed = 7

from sklearn.model_selection import KFold
kfold = KFold(num_folds, True, random_state=seed)

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

from sklearn.model_selection import cross_val_score
resultado = cross_val_score(model, x, y, cv=kfold)

print("Acurácia Final %.3f%%" % (resultado.mean() * 100.0))

# SVM - suport vector machine 

Objetivo desse algoritmo é buscar uma linha que melhor separa duas classes dentro de um conjunto de dados. As instãncias de dados que estão mais proximas dessa linha que separa as classes, são chamadas support vectors. O SVM tem sido entendido para suportar multiplas classes

Support Vector Machine são algoritmos de classificação  muito poderosos. Quando usados em conjunto com "Randon Forest" e outras ferramentas de aprendizagem de máquina, dão uma dimensão muito diferente para a montagem de modelos. Assim eles se tornam cruciais para os casos em que é necessária um poder de previsão muito elevado. Esses algoritmos são um pouco mais dificeis de visualizar devido a complexidade na formulação

In [ ]:
from pandas import read_csv

# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

num_folds = 5
seed = 7

from sklearn.model_selection import KFold
kfold = KFold(num_folds, True, random_state=seed)

from sklearn.svm import SVC
model = SVC()

from sklearn.model_selection import cross_val_score
resultado = cross_val_score(model, x, y, cv=kfold)

print("Acurácia Final %.3f%%" % (resultado.mean() * 100.0))

## Salvando o resultado do seu modelo

In [ ]:
import pickle
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

teste_size = 0.33
seed = 7

X_treino, X_test, Y_treino, Y_test, = train_test_split(x,y, test_size = teste_size, random_state = seed)

# criando modelo
modelo = LogisticRegression()

# treinando modelo
modelo.fit(X_treino, Y_treino)

# salvando modelo
arquivo = '9-Arquivos-Cap06/modelos/modelo_classificador_final2.sav'
pickle.dump(modelo, open(arquivo, 'wb'))

# carregando arquivo
modelo_classificador = pickle.load(open(arquivo, 'rb'))
modelo_prod = modelo_classificador.score(X_test, Y_test)
print('Modelo carregado')

# print do resultado
print("Acurácia %.3f" % (modelo_prod.mean() * 100))

## Algoritmo XGBosst - Extreme Gradient Bossting

O algoritmo XGBosst é uama extensão do GBM (Gradient Boosting Method) que permite trabalhar com multithreading em uma unica maquina e processamento paralelo em um cluster de varios servidores. A principal vsntagem do XGBoost sobre o GBM é sua capacidade de gerenciar dados esparsos.
O XGBoost automáticamente aceita dados esparsos como input sem armazenar zeros na memória

Principais vantagens do XGBosst

1 - aceita dados esparsos (oque permite trabalhar com patrizes esparsas), sem a necessidade de converão para matrizes densas. 

2 - constrói uma arvore de aprendizagem utilizando um moderno método de split (chamado quantile sketch), oque resulta em um tempo de processamento muito menor que métodos tradicionais. 

3 - Permite computação paralela em uma unica maquina (utilizando multithreading) e processamento paralalo em maquinas distribuidas em cluster.

Basicamente o XGboos utiliza os mesmo parâmetros que o GBM e permite tratamento avançado de dados missing

O XGBoost é muito utilizado por Cientista de dados que vencem competições do kaggle

Instalar o XGBoost através do Pypi

!pip install xgboost

In [ ]:
conda install py-xgboost

In [ ]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# carregando os dados
arquivo = '9-Arquivos-Cap06/data/pima-data.csv'
colunas = ['preg','plas','pres','skin','test','mass','pedi','age','class']
dados = read_csv(arquivo, names=colunas)
array = dados.values

# separando o array em componentes de imput (x) e output (y)
x = array[:,0:8]
y = array[:,8]

teste_size = 0.33
seed = 7

X_treino, X_test, Y_treino, Y_test, = train_test_split(x,y, test_size = teste_size, random_state = seed)

modelo = XGBClassifier()

# treinando modelo
modelo.fit(X_treino, Y_treino)

# print do modelo
print(modelo)

# fazendo previões
y_pred = modelo.predict(X_test)
previsoes = [round(value) for value in y_pred]

# avaliando as previsões
accuracy = accuracy_score(Y_test, previsoes)
print("Acurácia %.2f%%" % (accuracy * 100.0))